In [ ]:
import os, shutil            #Operating System
#from pyswmm import Simulation, raingages, Nodes, SystemStats, Links
import pandas as pd
from swmmio.utils.modify_model import replace_inp_section
from swmmio import create_dataframeINP
import swmmio
from swmmio.graphics import swmm_graphics as sg
import math
import numpy as np
import datetime as DT
from functions import *

In [ ]:
if os.getcwd() != '/home/fanda/Dropbox/doc/trento/Tesi/swmmio':
    os.chdir("./swmmio")
#os.listdir()


In [ ]:
BaseModel = swmmio.Model(r'../inp_file/rovereto_mod.inp')

swmmio.Model.subcatchments

# Crea i dataframe con le informazioni dell'inpfile originale
## RainGages
GAGES = create_dataframeINP(BaseModel.inp.path,'[RAINGAGES]')

## TimeSeries
TIMESERIES = create_dataframeINP(BaseModel.inp.path,'[TIMESERIES]')
TIMESERIES = pd.DataFrame( {'[TIMESERIES]' : TIMESERIES['[TIMESERIES]'].unique()} ) # Elimino i doppi
TIMESERIES = pd.DataFrame(TIMESERIES['[TIMESERIES]'].str.split().tolist(),
                          columns=['Name', 'Date', 'Time'],
                         )
TIMESERIES = TIMESERIES.set_index('Name')

## Nodi Dividers
DIVIDERS = create_dataframeINP(BaseModel.inp.path,'[DIVIDERS]')
DIVIDERS = pd.DataFrame(DIVIDERS['[DIVIDERS]'].str.split().tolist(), 
                        columns = ['Name','Elevation', 'DivertedLink', 
                                   'Type', 'CutoffFlow','MaxDepth', 
                                   'P1', 'P2', 'P3'], 
                                    )
DIVIDERS = DIVIDERS.set_index('Name')

NODES = BaseModel.nodes()
SUBCATCHMENTS = BaseModel.subcatchments()
CONDS = BaseModel.conduits()

# Definizione dei diametri commericiali
# https://www.oppo.it/tabelle/diametri_interni_tubi.htm#fibrocemento
DiametriCommericali = [.060,.080,.100,.125,
                       .150,.170,.200,.225,
                       .250,.300,.350,.400,
                       .450,.500,.600,.700,
                       .800,.900,1.00, 1.1,
                        1.2,1.3,1.4,1.5,1.6]



In [ ]:
## Crea le liste dei percorsi dai nodi di monte a quelli di valle

conds = BaseModel.conduits()
subcat = BaseModel.subcatchments()
Branchs = []
NodeDividerLost = []

for NodeIn in CONDS[CONDS.InletNode.isin(CONDS.OutletNode) == False ].InletNode:
    Chain = [NodeIn] # Inizializzo la Lista
    while isinstance(NodeIn, str):
        if any(DIVIDERS.index == NodeIn):  ## Il nodo è un Divider
            CondOut = DIVIDERS[DIVIDERS.index == NodeIn].DivertedLink[0]
            NodeOut = CONDS[CONDS.index == CondOut].OutletNode[0]

            OtherCondOut = CONDS[ ( CONDS.InletNode == NodeIn) & (CONDS.OutletNode != NodeOut) ] 
            OtherNodeOut = OtherCondOut.OutletNode[0]
            
            NodeDividerLost.append(OtherNodeOut)            
        else:
            try:
                CondOut = CONDS[ CONDS.InletNode == NodeIn ]
                NodeOut = CondOut.OutletNode[0]
            except:
                #print("fine coda")
                break
        NodeIn = NodeOut # Riassegnazione

        Chain.append(NodeOut)
    Branchs.append(Chain)

for NodeIn in list(set(NodeDividerLost)): 
    # I Nodi dopo un divider sono come i primi di un nuovo branch
    Chain = [NodeIn] # Inizializzo la Lista
    while isinstance(NodeIn, str):
        if any(DIVIDERS.index == NodeIn):  ## Il nodo è un Divider
            CondOut = DIVIDERS[DIVIDERS.index == NodeIn].DivertedLink[0]
            NodeOut = CONDS[CONDS.index == CondOut].OutletNode[0]

            OtherCondOut = CONDS [ ( CONDS.InletNode == NodeIn) & (CONDS.OutletNode != NodeOut) ] 
            OtherNodeOut = OtherCondOut.OutletNode[0]
            
            NodeDividerLost.append(OtherNodeOut)            
        else:
            try:
                CondOut = CONDS[ CONDS.InletNode == NodeIn ]
                NodeOut = CondOut.OutletNode[0]
            except:
                #print("fine coda")
                break
        NodeIn = NodeOut # Riassegnazione

        Chain.append(NodeOut)
    Branchs.append(Chain)
    
df_Branches = pd.DataFrame()
df_Branches['Branches'] = Branchs

Branches_cond = []
for branch in df_Branches.Branches.tolist():
    branchCond = []
    for node in branch[:-1]:
        Index = branch.index(node)
        NextIndex = Index + 1
        NodeIn  = branch[Index]
        NodeOut = branch[NextIndex]
        cond = CONDS[ (CONDS.InletNode == NodeIn) & (CONDS.OutletNode == NodeOut)].index[0]
        branchCond.append(cond)
    Branches_cond.append(branchCond)

df_Branches['BranchesCond'] = Branches_cond

df_Branches

In [ ]:
def BranchsAfferenti(NODE, Branchs = df_Branches, Conds = CONDS):
    
    BranchesList = []
    
    for branch in Branchs['Branches']:
        #print(branch)
        try:
            index = branch.index(NODE)
        except ValueError:
            continue
        BranchesList.append(branch[0:index+1])
    
   # print(BranchesList)
    IsLeaf = True
    for branch in BranchesList:
        if len(branch) == 1:
            IsLeaf *= True
        else: 
            IsLeaf *= False
    if IsLeaf: return('Leaf')
        

    CondotteAfferenti = []
    CondottaSuccessiva = []
    
    for branch in BranchesList:
        cond = Conds[ (Conds.InletNode == branch[-2]) & (Conds.OutletNode == NODE)].index[0]
        CondotteAfferenti.append(cond)
        try:
            NextCond = Conds[ ( Conds.InletNode == NODE ) & (Conds.OutletNode == branch[-1])][0]
        except KeyError:
            NextCond = None
        CondottaSuccessiva.append(NextCond)
        
    CondotteAfferenti = list(set(CondotteAfferenti))
    
    if any(DIVIDERS.index == NODE):  ## Il nodo è un Divider
        CondottaSuccessiva = DIVIDERS.DivertedLink[DIVIDERS.index == NODE][0]
    else:
        try:
            CondottaSuccessiva = Conds.index[Conds.InletNode == NODE ][0]
        except IndexError:
            print('Condotta di Out')
            CondottaSuccessiva = None
   # print(CondottaSuccessiva)
        

  #  print('Condotte afferenti al nodo '+ NODE + ': ' + str(CondotteAfferenti))
   # print('Condotta Successiva, da dimensionare: '  + str(CondottaSuccessiva))    
    
    LunghezzaPrincipale = []
    PercorsoPrincipale = []
    LunghezzaTotaleCondotte = []
    PendenzaMediaBacino = []
    TempoDiCorrivazione = []
    
    for cond in CondotteAfferenti:

        lengthBrantch = []
        AllConds = []
        for branch in df_Branches.BranchesCond.tolist():
            
            try:
                index = branch.index(cond)
            except ValueError:
                continue

            chain = branch[0:index+1]
            
            AllConds = AllConds + chain

            #print(chain)
            length = 0
            for cond in chain:
                length += Conds[Conds.index == cond].Length[0]                
                
            lengthBrantch.append(length)
            if max(lengthBrantch) == length:
                PP = chain
            #print(length)
        AllConds = list(set(AllConds))
        
        lengthAllConds = 0 
        pendConds = 0
        LungErr = 0
        for cond in AllConds:
            Prop = PropCond(cond, NODES, Conds, DIVIDERS)
            
            length = Prop['Length']
            lengthAllConds += length
            pend = Prop['Pendenza'] 
            pendConds += pend * length  # Per fare la media ponderata
            
        t0 = 0
        for cond in PP:
            #print(cond)
            Prop = PropCond(cond, NODES, Conds, DIVIDERS)
            t0 += Prop['Length'] / Prop['Velocity05']
            
        #print(t0)
        
        PendenzaMediaBacino.append(pendConds/ lengthAllConds)
        PercorsoPrincipale.append(PP)
        TempoDiCorrivazione.append(t0)
        MaxLength = max(lengthBrantch)
        LunghezzaPrincipale.append(MaxLength)
        LunghezzaTotaleCondotte.append(lengthAllConds)

   # print(PercorsoPrincipale)
   # print(LunghezzaPrincipale)
   # print(LunghezzaTotaleCondotte)
    
    NodeSubcatches = []
    for Cond in CondotteAfferenti:
        node = Conds[Conds.index == Cond].InletNode[0]
        NodeSubcat = []
        for branch in BranchesList:
            if node == branch[-2]:
                NodeSubcat = NodeSubcat + list(branch)
        NodeSubcat = list(set(NodeSubcat))
        NodeSubcatches.append(NodeSubcat)
  #  print(NodeSubcatches)
    
    Areas = []
    PercImperv = []
    for subcat in NodeSubcatches:
        Area = 0
        percimperv = 0
        for node in subcat:
            if node == NODE: 
                continue
            try:
                area = SUBCATCHMENTS[SUBCATCHMENTS.Outlet == node].Area[0]
                CoefImpermeab = SUBCATCHMENTS[SUBCATCHMENTS.Outlet == node].PercImperv[0] * area
            except IndexError:
                continue
            Area += area
            percimperv += CoefImpermeab
        if Area == np.nan:
            Area = 0
        Areas.append(Area)
      #  print(Areas)
        
        if Area != 0:
            PercImperv.append(percimperv/Area)
        else: 
            PercImperv.append(0.0)
    
    df = pd.DataFrame({ 'CondIn' : CondotteAfferenti, 
                        'LunghezzaPercorsoPrincipale' : LunghezzaPrincipale,
                        'AreaSottoBacino' :  Areas, 
                        'LunghezzaTotaleCondotte' : LunghezzaTotaleCondotte,
                        'PercImperv' : PercImperv,
                        'PendenzaMedia' : PendenzaMediaBacino,
                        'TempoDiCorrivazione' : TempoDiCorrivazione,
                       }) 
    # Calcolo Di lambda con la formula empirica di Ciaponi & Papiri [1992]
    A = df['AreaSottoBacino']
    d = df['LunghezzaTotaleCondotte'] / A # coef corretivo : [m] / [ha] 
    I = df['PercImperv'] / 100
    S = df['PendenzaMedia']
   # print(A)
  #  print(d)
    df['Lambda'] = .5 * A ** .351 * d ** .358 * I ** -.163 * S ** -.29   
    df['Lambda'] = df['Lambda'].fillna(0)
    
    ddf = pd.DataFrame()
    
    return({'df' :df,
           'CondottaSuccessiva' : CondottaSuccessiva,
           })
    
def SubcatAfferenti(NODE, Conds = CONDS):
    if any(DIVIDERS.index == NODE):  ## Il nodo è un Divider
        NextCond = DIVIDERS.DivertedLink[DIVIDERS.index == NODE][0]
    else:
        try:
            NextCond = Conds.index[Conds.InletNode == NODE ][0]
        except IndexError:
            return
 #   print(NextCond)
    try :
        Subcat = SUBCATCHMENTS[SUBCATCHMENTS.Outlet == NODE]    
        A = Subcat.Area[0]
        PercImperv = Subcat.PercImperv[0]
    except IndexError:
        A = 0
        PercImperv = 0
    
    propNextCond = PropCond(NextCond, NODES, Conds, DIVIDERS)
    
    Length = propNextCond['Length']
    d = Length / A
    I = PercImperv / 100
    S = propNextCond['Pendenza']
    Ks = propNextCond['Ks']
   # print(S)
    if A != 0:
        Lambda =  .5 * A ** .351 * d ** .358 * I ** -.163 * S ** -.29   
    else:
        Lambda = 0
    df = {  'CondOut'    : NextCond, 
            'Length'     : Length,
            'Area'       : A, 
            'Ks'         : Ks, 
            'PercImperv' : PercImperv,
            'Pendenza'   : S,
            'Lambda'     : Lambda,
         }
    return(df)
    
    
    
df = BranchsAfferenti('J121')
df

ddf = SubcatAfferenti('J506')


df['df']
print(df)

ddf


In [ ]:
def Qexp(A,b,j,phi):
    return phi*A*j*b

def expIUH(t,tk,tp,ll):
    if ll == 0:
        return(0.0)
    if t < tk:
        return(0.0)
    elif t <=  tp + tk :
        return(1 - np.exp(-(t-tk)/ll))
    else:
        return((np.exp(tp/ll) - 1)*np.exp(-(t-tk)/ll))
    
def EXPIUH_DF(row):
   # print(row)
    return( expIUH( t = row['t'], tk = row['tk'], tp = row['tp'], ll = row['ll']) )

def SpeedCond(Ks,D,G,i):
    Angle = theta(G)
    #print('Angle :' + str(Angle))
    Rh = D/4 * (1-math.sin(Angle)/Angle)
#    print(Rh)
    Speed = Ks * (Rh ** (2/3)) * (i **.5)
   # print(Speed)
    return(Speed)
 

In [ ]:
def distanceNodes(node1, node2, Branchs = df_Branches, Conds = CONDS):
    #BranchesList = []
    for branch in Branchs['Branches']:
        #print(branch)
        try:
            index1 = branch.index(node1)
            index2 = branch.index(node2)
        except ValueError:
            continue
        course = branch[index1:index2+1]
    print(course)
    
    CondList = []
    LengthList = []
    for node in course:
        index = course.index(node)
        try: 
            InNode = course[index]
            OutNode = course[index+1]
        except:
            break
        cond = Conds[ (Conds.InletNode == InNode) & (Conds.OutletNode == OutNode)].index[0]
        length = Conds[Conds.index == cond].Length[0]
        CondList.append(cond)
        LengthList.append(length)
    
    df = pd.DataFrame({'conds' : CondList, 'length' : LengthList})
    print(df)
    
    return(df.length.sum())
    
distanceNodes('J507','J121')

In [ ]:
#for node in ['J166']:
NewDiameters = []

aLSPP = 36.253816 # che derivano dall'analisi pluviometrica
nLSPP = 0.299421 

for node in CONDS.InletNode.tolist():
#for node in ['J384']:
    DATAFRAMECOMPLEX = pd.DataFrame()
    b = 10/3.6 # per avere Q in l/s
    Branch = BranchsAfferenti(node, Branchs = df_Branches)
    
    if Branch == 'Leaf': 
        print('Leaf')
        df = None
    else:
        df = Branch['df']
    NextCond = SubcatAfferenti(node, Conds = CONDS)['CondOut']

    if NextCond == None: 
        ## é nodo di out
        break
        
    propCond = PropCond(NextCond, NODES, CONDS, DIVIDERS)
    Ks = propCond['Ks']
    i  = propCond['Pendenza']
    #  print(propCond)
    D  = propCond['Diametro'] #Diametro di primo tentativo
    Length = propCond['Length']
    LambdaOut = SubcatAfferenti(node, Conds = CONDS)['Lambda']
    Area = SubcatAfferenti(node, Conds = CONDS)['Area']
    PercImperv = SubcatAfferenti(node, Conds = CONDS)['PercImperv']
    
    NewDiameter = 20.
    
    while D != NewDiameter: # criterio di convergenza
        Qmax = 0.
        D = NewDiameter
        speedCond = SpeedCond(Ks = Ks, 
                              D  = D,
                              G  = 0.8,
                              i  = i) # m/s
       # print(speedCond)
        
        if Branch == 'Leaf':
            RStar = rStar(nLSPP)
           # print('R*:', RStar)
            ll = LambdaOut
            tpp = RStar * ll
           # print('tp:',tpp)
            j = aLSPP * (tpp/60)**(nLSPP - 1) ## mm/h
            Qmax = Qexp(Area,b,j,PercImperv / 100) * (1-np.exp(-tpp/ll))
        else:  
            for tp in range(5,45,5):
                Qi = []
                tpp = tp
                j = aLSPP * (tp/60)**(nLSPP - 1) ## mm/h

                A = df.AreaSottoBacino
                phi = df.PercImperv
                #print(phi)
                Qconst = Qexp(A,b,j,phi/100)
                Qconst = Qconst.fillna(0)

                ll = df.Lambda 
                #print(ll)
                tk = ll*0 + Length / speedCond / 60  # in Munuti

                tp = ll*0+tp
                tk = tk.fillna(0)
               # print(tk)
                #NextCond = SubcatAfferenti(node, Conds = CONDS)
            
                Qc = Qexp(Area, b, j, PercImperv/100)  #l/s
            
                for t in range(0,45,1):
                    tt = t
                    if Branch != 'Leaf':
                        t = ll*0+t

                        ddf = pd.DataFrame({'t':t, 'tk':tk, 'tp': tp, 'll': ll})
                        #print(ddf)
                        EXPIUH = ddf.apply(EXPIUH_DF,axis=1)

                        #print((Qconst*EXPIUH).tolist())
                        tmpQi = (Qconst*EXPIUH).tolist() 
                    else:
                        tmpQi = []

                    # Idrogramma Sottoarea direttamente afferente

                    expiuh = expIUH(tt,0.,tpp,LambdaOut)

                    tmpQi.append( Qc * expiuh )

                    Qi.append(tmpQi)
            #        print(tmpQi)
                    #print(tt, tpp, expiuh,Qc, NextCond)
                Qi = pd.DataFrame(Qi)
                DATAFRAMECOMPLEX[tpp] = Qi.sum(axis=1)
               # graf = Qi.plot(title = 'Node:'+ node+ ' tp:' + str(tpp))

                #print(Qi)
                Qpeak = max(Qi.sum(axis=1))
                if Qmax < Qpeak:
                    Qmax = Qpeak
                #print(tpp,Qpeak)
        print('Qmax =',Qmax)
        
        TetaOpt = theta(0.8)

        DiametroOttimale =  10**(-9/8) * ( 2 ** (13/3) * Qmax / Ks / (i ** .5) *   \
                  (1 - math.sin(TetaOpt) / TetaOpt) ** (-2/3) * \
                  (TetaOpt - math.sin(TetaOpt)) ** -1 ) ** (3/8)
       # print('DiametroOpt =',DiametroOttimale)
        NewDiameter = round2list(DiametroOttimale, DiametriCommericali)
        
    # Controllo criterio di autopulizia
    # i >= 2/gamma Rh 
    if i < 2/1000/(NewDiameter/4*(1  - np.sin(TetaOpt)/TetaOpt)) :
        print('Pendenza Bassa, non rispetta il criterio di autopulizia')

    print('Diametro nodo',node,'=',NewDiameter)
    NewDiameters.append([NextCond, NewDiameter])
    if Branch != 'Leaf':
        plt = DATAFRAMECOMPLEX.plot(title = 'Cond:'+ NextCond )
        plt.set_xlabel('t [min]')
        plt.set_ylabel('Q [l/sec]')
        fig = plt.get_figure()
        fig.savefig(node + '_Qmax.png', dpi = 300)

NNNDIAMM = pd.DataFrame(NewDiameters)
NNNDIAMM = NNNDIAMM.set_index(0)
NNNDIAMM = NNNDIAMM.rename(columns={1: 'DiametriNuovi'})
NNNDIAMM.to_csv('DiametriProgettati.csv')
NNNDIAMM



In [ ]:
NNNDIAMM

In [ ]:
NNNDIAMM

In [ ]:
SUBCATCHMENTS[SUBCATCHMENTS.Outlet == 'J507'].T



In [ ]:
CONDS.index[(CONDS.InletNode == 'J506') & (CONDS.OutletNode == 'J122' )]

In [ ]:
df.Lambda *3 

In [ ]:
conds.Geom1.to_csv('DiametriAttuali')

In [ ]:
PropCond('C1221',NODES, CONDS,DIVIDERS)['Length']

In [ ]:
aa = pd.DataFrame([[1,2],[5,4]])

In [ ]:
aa = aa.append([1,2])


In [ ]:
aa